In [123]:
import uproot
import matplotlib.pyplot as plt
import numpy as np
import hist as Hist
import awkward as ak

In [23]:
def get_adjacent_strips(pixel, pixels_per_plane = 200, n = 2):
    '''
    Returns n adjacent strips within the same plane from a given strip (pixel). 
    N is determined by the spline width and can be adjusted as necessary. 
    Assumed pixel pitch of 200 microns and a spline width of 600 microns.
    '''
    return [(pixel + x) for x in range(-n, n+1) if np.ceil(float((pixel + x)/pixels_per_plane)) == np.ceil(float(pixel/pixels_per_plane))]

In [ ]:
n = 2
pixel = 100
[(pixel + x) for x in range(-n, n+1)]

In [ ]:
get_adjacent_strips(199)

In [ ]:
#Seeing the file for the histogram

with uproot.open('./BNL_Signal_Response.root') as infile:
    h = infile['pmax_histogram'].values()
    bruh = [i for i in h]
    print(bruh)





In [ ]:
#Test of class

import matplotlib.pyplot as plt
import numpy as np
import uproot  
from energy_sharingv2 import digitizeatar

path = "./BNL_Signal_Response.root"

f = uproot.open(path)

h = f['pmax_histogram'].to_hist()

x = digitizeatar(path)

out = x.build_spline()
xs = np.linspace(h.axes[0].centers[0], h.axes[0].centers[-1], 1000)



ys = x.build_spline()(xs)

fig, ax = plt.subplots()

plt.plot(xs, ys)

In [28]:
#Build an algorithm for energy sharing
infile = uproot.open('/home/dylan/pioneer/simulationdata/pienux_out_stripped.root:atar')
data = infile.arrays(['pixel_pdg', 'pixel_edep', 'pixel_time', 'pixel_hits'])

pdg = data['pixel_pdg']
edep = data['pixel_edep']
time = data['pixel_time']
hits = data['pixel_hits']

pixelpitch = 200


energydata  = edep[0]
hitdata = hits[0]

def compute_energy_sharing_algorithm(data, spline, entry = None):  
    for k in energydata:

        for i, hit in enumerate(hitdata):

            these_energies = []
            
            places = get_adjacent_strips(hitdata[i])

            for pos in places:

                pix_diff = hit - pos
                energy_position = pix_diff * pixelpitch
                energyi = spline(energy_position) * energydata[i]

                these_energies.append(energyi)

            these_energies = np.array(these_energies) * (energydata[i] * np.sum(these_energies))

            


from energy_sharingv2 import digitizeatar

path = "./BNL_Signal_Response.root"

x = digitizeatar(path)
spline = x.build_spline()

compute_energy_sharing_algorithm(energydata, spline)


        

In [83]:
updated_infile = uproot.recreate('/home/dylan/pioneer/simulationdata/pienux_out_stripped.root:atar')

updated_infile['tree'] = uproot.newtree({'branch1':int})
updated_infile['tree'].extend({'branch1': np.array([1, 2, 3, 4, 5])})



AttributeError: module 'uproot' has no attribute 'newtree'

In [125]:
# createdfile = uproot.create('./testfile.root')

with uproot.update('./testfile.root') as testfile:
    testfile['hist'] = np.histogram(np.random.normal(0, 1, 100000))
    testfile['tree'] = {'branch':ak.Array}

testfile.keys()


TypeError: unrecognized type cannot be written to a ROOT file: dict

In [126]:
with uproot.open('./testfile.root') as f:
    h = f["hist"]
    print(h)
    


<TH1D (version 3) at 0x7f8e85116e20>


In [151]:
with uproot.recreate('./testfile.root') as f:
    f.mkdir('data')
    f['data'] = {'branch': ak.Array([1, 2, 3])}

    print(f.keys())
  


['data;1', 'data;2']


In [148]:
infile = uproot.open('./testfile.root')
infile.keys()

['data;1', 'data;2']

In [158]:
tfile = uproot.open('./testfile.root')
tfile['data']['branch'].array()

<Array [1, 2, 3] type='3 * int64'>

In [118]:
tfile = uproot.open('/home/dylan/pioneer/simulationdata/pienux_out_stripped.root:atar')
tfile.show()

name                 | typename                 | interpretation                
---------------------+--------------------------+-------------------------------
pixel_hits           | std::vector<int32_t>     | AsJagged(AsDtype('>i4'), he...
pixel_pdg            | std::vector<int32_t>     | AsJagged(AsDtype('>i4'), he...
pixel_edep           | std::vector<double>      | AsJagged(AsDtype('>f8'), he...
pixel_time           | std::vector<double>      | AsJagged(AsDtype('>f8'), he...
nhit                 | int32_t                  | AsDtype('>i4')
has_pion             | bool                     | AsDtype('bool')
pion_decay           | bool                     | AsDtype('bool')
pion_dar             | bool                     | AsDtype('bool')
has_muon             | bool                     | AsDtype('bool')
muon_decay           | bool                     | AsDtype('bool')
muon_dar             | bool                     | AsDtype('bool')
has_positron         | bool                     | AsD

In [159]:
#Build an algorithm for energy sharing
infile = uproot.open('/home/dylan/pioneer/simulationdata/pienux_out_stripped.root:atar')
data = infile.arrays(['pixel_pdg', 'pixel_edep', 'pixel_time', 'pixel_hits'])

pdg = data['pixel_pdg']
edep = data['pixel_edep']
time = data['pixel_time']
hits = data['pixel_hits']

pixelpitch = 200


energydata  = edep[0]
hitdata = hits[0]

def compute_energy_sharing_algorithm(data, spline, entry = None): 

    output = {"pdg":}

    for k in energydata:

        for i, hit in enumerate(hitdata):

            these_energies = []
            
            places = get_adjacent_strips(hitdata[i])

            for pos in places:

                pix_diff = hit - pos
                energy_position = pix_diff * pixelpitch
                energyi = spline(energy_position) * energydata[i]

                these_energies.append(energyi)

            these_energies = np.array(these_energies) * (energydata[i] * np.sum(these_energies))

    with uproot.recreate('/home/dylan/pioneer/simulationdata/pienux_out_stripped.root:atar') as f:
        

            


from energy_sharingv2 import digitizeatar

path = "./BNL_Signal_Response.root"

x = digitizeatar(path)
spline = x.build_spline()

compute_energy_sharing_algorithm(energydata, spline)


        

In [180]:
from energy_sharingv2 import digitizeatar

path = "./BNL_Signal_Response.root"

x = digitizeatar(path)

spline = x.build_spline()


atar_data = uproot.open('/home/dylan/pioneer/simulationdata/pienux_out_stripped.root:atar')
x.compute_energy_sharing(atar_data, spline, 0)

ValueError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`), and integer or boolean arrays (possibly jagged) are valid indices

(https://github.com/scikit-hep/awkward-1.0/blob/1.7.0/src/python/content.cpp#L694)